In [4]:
!python3 -m pip install google-cloud-aiplatform --upgrade

     |████████████████████████████████| 1.8 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 41.8 MB/s eta 0:00:01
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.30.0
    Uninstalling google-cloud-storage-1.30.0:
      Successfully uninstalled google-cloud-storage-1.30.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 0.6.0
    Uninstalling google-cloud-aiplatform-0.6.0:
      Successfully uninstalled google-cloud-aiplatform-0.6.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

google-cloud-pipeline-components 0.1.0 requires google-cloud-aiplatform<1.0.0,>=0.7.1, but you'll have google-cloud-aiplatform 1.0.1 whic

In [16]:
project_id = 'pytorch-tpu-nfs'
staging_bucket = 'gs://automl-samples'
location='us-central1'
context_window = 24
run_id = f'context-window-{context_window}'
experiment_prefix = 'weather-exp'


In [17]:
from google.cloud import aiplatform
from datetime import datetime

# Create and experiment tag
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
experiment_id = experiment_prefix + TIMESTAMP

# Init AI Platform
aiplatform.init(
    project=project_id,
    staging_bucket=staging_bucket,
    experiment=experiment_id
)

# Define the custom training job
job = aiplatform.CustomContainerTrainingJob(
    display_name="view-training",
    container_uri='gcr.io/pytorch-tpu-nfs/test-custom-trainer:latest',
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)


model = job.run(
    replica_count=1, 
    model_display_name="temp-prediction",
    args=[
        f'--experiment-id={experiment_id}', 
        f'--staging-bucket={staging_bucket}',
        f'--context-window={context_window}'
    ]
)
metrics_df = aiplatform.get_experiment_df(experiment_id)
#metrics_df.loc[exp_df['run_name'] == run_id]['metric.val_mae'].values[-1]

print(metrics_df)
out_model = model

INFO:root:Resource weather-exp20210608042841 not found.
INFO:root:Creating Resource weather-exp20210608042841
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://automl-samples/aiplatform-custom-training-2021-06-08-04:28:46.366 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6371894183333986304?project=64701051322
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/64701051322/locations/us-central1/trainingPipelines/6371894183333986304 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/64701051322/locations/us-central1/trainingPipelines/6371894183333986304 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/64701051322/locations/us-central1/trainingPipelines/6371894183333986304 current state:
Pi